In [1]:
import numpy as np
import os 

from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score 

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense 
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam

## Define Constants

In [2]:
from utils.config import LABEL_MAP, DATA_LEN, DATA_PATH, NUM_EXAMPLES, SEQUENCE_LENGTH, ACTIONS, TEST_SPLIT, NUM_EPOCHS

## Get Data from Data Folder

In [3]:
# loop throuh each action folder, collect the example data and the appropraite label (enusre data has been created)

examples, labels = [], []
for action in ACTIONS: 
    for sequence in range(NUM_EXAMPLES):
        
        # load in all numpy data for a given example 
        window = []
        for frame in range(SEQUENCE_LENGTH):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame)))
            window.append(res)
        
        # for current example, add all data for that sequence and appropraite label 
        examples.append(window)
        labels.append(LABEL_MAP[action])

In [4]:
# convert data into appropraite format
X, y = np.array(examples), to_categorical(labels).astype(int)

In [5]:
# Split our data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SPLIT) # 5% to test and 95% to train 

In [10]:
# web app that allows you to see training
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir Logs

## Build the model

In [7]:
# build model architecture 
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape = (SEQUENCE_LENGTH, DATA_LEN))) # to stack LSTMs return_sequences needs to be True, the next layer will recieve the "history"
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(ACTIONS.shape[0], activation='softmax'))


In [11]:
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596675 (2.28 MB)
Trainable params: 59667

## Train Model and Save it

In [12]:
model.fit(X_train, y_train, epochs=NUM_EPOCHS, callbacks=[tb_callback])

Epoch 1/250
3/3 [==============================] - 5s 134ms/step - loss: 1.0962 - categorical_accuracy: 0.2353
Epoch 2/250
3/3 [==============================] - 0s 131ms/step - loss: 1.0856 - categorical_accuracy: 0.3647
Epoch 3/250
3/3 [==============================] - 0s 153ms/step - loss: 1.0722 - categorical_accuracy: 0.3529
Epoch 4/250
3/3 [==============================] - 0s 163ms/step - loss: 1.0555 - categorical_accuracy: 0.3529
Epoch 5/250
3/3 [==============================] - 1s 175ms/step - loss: 1.0375 - categorical_accuracy: 0.3529
Epoch 6/250
3/3 [==============================] - 1s 162ms/step - loss: 1.0040 - categorical_accuracy: 0.4706
Epoch 7/250
3/3 [==============================] - 1s 172ms/step - loss: 0.9315 - categorical_accuracy: 0.5412
Epoch 8/250
3/3 [==============================] - 0s 142ms/step - loss: 0.8625 - categorical_accuracy: 0.5882
Epoch 9/250
3/3 [==============================] - 0s 162ms/step - loss: 0.7479 - categorical_accuracy: 0.6118
E

KeyboardInterrupt: 

In [13]:
# Save the model, you can name it whatever you want
model.save('Models/gesture.keras')

## Evaluate Model

In [14]:
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

accuracy_score(ytrue, yhat)

1/1 [==============================] - 1s 650ms/step


0.8

In [15]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[2, 0],
        [1, 2]],

       [[2, 1],
        [0, 2]]], dtype=int64)